In [1]:
from itertools import product
import pandas as pd
import statsmodels.tsa.statespace.sarimax as sm
import numpy as np
import ray
from numba import jit
import os
from collections import defaultdict
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [2]:
#@ray.remote
def fit_sarmiax(data, param, s):
    model=sm.SARIMAX(data, order=(param[0], param[1], param[2]),
                                            seasonal_order=(param[3], param[4], param[5], s)).fit(disp=-1)
    return model

In [3]:
def optimizeSARIMA(data, parameters_list, s):
    """
        data - data to modelling
        parameters_list - list with (p, q, P, Q) tuples
        d - integration order in ARIMA model
        D - seasonal integration order
        s - length of season
    """
   
    results = []
    params = []
    i = 1
    for param in parameters_list:
        print('Tested {} from {}'.format(i, len(parameter_list)))
        i+=1
        try:
            model= fit_sarmiax(data, param, s)
            results.append(model.aic)
            params.append(param)
        except Exception as err:
            continue
    #results = ray.get(results)
    #aic = [model.aic for model in results]
    d = {'aic': results, 'paramters':parameters_list}
    result_table = pd.DataFrame(d)
    # sort, lower AIC crit -> better
    result_table = result_table.sort_values(by='aic', ascending=True).reset_index(drop=True)
    return result_table

In [4]:
def fitSARIMAX(datas, column, installations, parametes, s):
    #results = []
    arparams = []
    maparams = []
    seasonalarparams = []
    seasonalmaparams = []
    i = 1
    for installation in installations:
        print('Fitted {} from {}'.format(i, len(installations)))
        tmp = datas[installation]
        model = fit_sarmiax(tmp[column], parameters, s)
        arparams.append(model.arparams)
        maparams.append(model.maparams)
        seasonalarparams.append(0)
        #seasonalarparams.append(model.seasonalarparams)
        seasonalmaparams.append(model.seasonalmaparams)
        #results.append(model)
        i+=1
    #results = ray.get(results)
    #d = {'installations': installations, 'model': results}
    return arparams, maparams, seasonalarparams, seasonalmaparams

In [5]:
def prepare_data(path):
    data = pd.read_csv(os.path.join(path, 'airly_data.csv'))
    data.start_date = pd.to_datetime(data.start_date)
    data = data.sort_values(by='start_date').reset_index(drop=True)
    return data

In [6]:
def fit_data(installation, df):
    data = df.copy()
    if installation == 'global':
        data = data.groupby(data.start_date).mean().drop(['installation_id'], 1).sort_index().reset_index(drop=True)
        return data
    else:
        data = data[data.installation_id == installation].set_index('start_date').sort_index().reset_index(drop=True)
        return data

In [7]:
def installations_data(data, installations):
    datas = defaultdict()
    for installation in installations:
        datas[installation] = fit_data(installation, data)
    return datas

In [8]:
def installations(data):
    return data.installation_id.drop_duplicates().tolist()

In [9]:
#ray.init()

In [10]:
ps = range(1, 4)
ds = range(0, 2)
qs = range(0, 3)
Ps = range(0, 3)
Ds = range(0, 2)
Qs = range(0, 3)
s =  24
# ps = range(1, 2)
# ds = range(0, 1)
# qs = range(1, 2)
# Ps = range(1, 2)
# Ds = range(0, 1)
# Qs = range(1, 2)
# s =  24
parameter_list = list(product(ps, ds, qs, Ps, Ds, Qs))
len(parameter_list)

324

In [11]:
data = prepare_data('/home/grzegorz/Academia/lectures/Longitudinal_and_Functional_Data_Analysis')

In [12]:
df = fit_data('global', data)

In [ ]:
%time result_table = optimizeSARIMA(df.PM10, parameter_list, s)

In [13]:
# parameters = result_table.iloc[0, 1]
parameters = (3, 0, 1, 0, 1, 1)
parameters

(3, 0, 1, 0, 1, 1)

In [14]:
installations_ids = installations(data)
count = data.groupby('installation_id').count().sort_values(by='PM10')
count = count[count.PM10 > 1000]
count = count.index.tolist()
installations_ids = list(set(installations_ids).intersection(set(count)))
len(installations_ids)

758

In [15]:
datas = installations_data(data, installations_ids)

In [16]:
%time arparams, maparams, seasonalarparams, seasonalmaparams = fitSARIMAX(datas, 'PM10', installations_ids, parameters, s)

Fitted 1 from 758
Fitted 2 from 758
Fitted 3 from 758
Fitted 4 from 758
Fitted 5 from 758
Fitted 6 from 758
Fitted 7 from 758
Fitted 8 from 758
Fitted 9 from 758
Fitted 10 from 758
Fitted 11 from 758
Fitted 12 from 758


/home/grzegorz/anaconda3/envs/hpc/lib/python3.6/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Fitted 13 from 758
Fitted 14 from 758
Fitted 15 from 758
Fitted 16 from 758
Fitted 17 from 758
Fitted 18 from 758
Fitted 19 from 758
Fitted 20 from 758
Fitted 21 from 758
Fitted 22 from 758
Fitted 23 from 758
Fitted 24 from 758
Fitted 25 from 758
Fitted 26 from 758
Fitted 27 from 758
Fitted 28 from 758
Fitted 29 from 758


/home/grzegorz/anaconda3/envs/hpc/lib/python3.6/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Fitted 30 from 758
Fitted 31 from 758
Fitted 32 from 758
Fitted 33 from 758
Fitted 34 from 758
Fitted 35 from 758
Fitted 36 from 758
Fitted 37 from 758
Fitted 38 from 758
Fitted 39 from 758
Fitted 40 from 758
Fitted 41 from 758
Fitted 42 from 758
Fitted 43 from 758
Fitted 44 from 758
Fitted 45 from 758
Fitted 46 from 758
Fitted 47 from 758
Fitted 48 from 758
Fitted 49 from 758
Fitted 50 from 758
Fitted 51 from 758
Fitted 52 from 758
Fitted 53 from 758
Fitted 54 from 758
Fitted 55 from 758
Fitted 56 from 758
Fitted 57 from 758
Fitted 58 from 758
Fitted 59 from 758
Fitted 60 from 758


...

Fitted 756 from 758


/home/grzegorz/anaconda3/envs/hpc/lib/python3.6/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Fitted 757 from 758


/home/grzegorz/anaconda3/envs/hpc/lib/python3.6/site-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Fitted 758 from 758
CPU times: user 13h 15min 4s, sys: 33.9 s, total: 13h 15min 38s
Wall time: 2h 12min 38s


In [17]:
#ray.shutdown()

In [18]:
# arparams = [models[models.installations == installation].iloc[0, 1].arparams for installation in installations_ids]
# maparams = [models[models.installations == installation].iloc[0, 1].maparams for installation in installations_ids]
# seasonalarparams = [models[models.installations == installation].iloc[0, 1].seasonalarparams for installation in installations_ids]
# seasonalmaparams = [models[models.installations == installation].iloc[0, 1].seasonalmaparams for installation in installations_ids]

In [19]:
coeffs = defaultdict()
coeffs['installation'] = installations_ids
for i in range(parameters[0]):
    coeffs['arparams_' + str(i)] = [arparams[j][i] for j in range(len(arparams))]
for i in range(parameters[2]):
    coeffs['maparams_' + str(i)] = [maparams[j][i] for j in range(len(maparams))]
for i in range(parameters[3]):
    coeffs['seasonalarparams_' + str(i)] = [seasonalarparams[j][i] for j in range(len(seasonalarparams))]
for i in range(parameters[5]):
    coeffs['seasonalmaparams_' + str(i)] = [seasonalmaparams[j][i] for j in range(len(seasonalmaparams))]
coeffs = pd.DataFrame(coeffs)
coeffs.to_csv('/home/grzegorz/Academia/lectures/Longitudinal_and_Functional_Data_Analysis/models_PM10.csv')

In [20]:
scaler = StandardScaler()

In [21]:
coeff_np = scaler.fit_transform(np.array(coeffs.iloc[:, 1:]))
epss = [2**i for i in range(-5, 5)]

In [22]:
clustering = []
for eps in epss:
    clustering.append(DBSCAN(eps=eps, min_samples=5, algorithm='brute', n_jobs=-1).fit_predict(coeff_np))